In [1]:
from sklearn.datasets import make_regression

In [2]:
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
# http://ufldl.stanford.edu/tutorial/supervised/MultiLayerNeuralNetworks/

### Generate data

In [5]:
rng = np.random.RandomState(0)

In [6]:
n_samples, n_features = 1000, 20

In [7]:
X, y = make_regression(n_samples, n_features, random_state=rng)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=rng)

### Baseline

In [9]:
from sklearn.linear_model import Lasso, Ridge

In [10]:
# reg = Lasso()
reg = Ridge(alpha=.5)

In [11]:
reg.fit(X_train, y_train)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [12]:
print(reg.score(X_test, y_test))

0.999999399117411


In [13]:
y_pred=reg.predict(X_test)

In [14]:
def r2_score(y_test,y_pred):
    u = ((y_test - y_pred) ** 2).sum()
    v = ((y_test - y_test.mean()) ** 2).sum()
    score = (1 - u/v)
    return score

In [15]:
r2_score(y_test,y_pred)

0.9999993991174111

In [16]:
from sklearn.metrics import mean_squared_error

In [17]:
mean_squared_error(y_test,y_pred)

0.027717268116372908

### MLP

In [18]:
# mu =  X_train.mean(0)
# std = X_train.std(0)
# X_train = np.divide(X_train - mu, std)
# X_test=np.divide(X_test - mu, std)

In [19]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [20]:
def tangent(z):
    return (np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z))

In [21]:
def leaky_rectified_linear(z):
    return z if z > 0 else 0.01*z

In [22]:
def rectified_linear(z):
    return np.maximum(0,z)

In [23]:
def MLP_4layers(X, y, epochs = 1000, lambda_ =0.01, learning_rate=1e-3, units_l2=15, units_l3=15, units_l4=1):
    
    units_l1 = X.shape[1]
    
    np.random.seed(0)
    
    W1=np.random.rand(units_l2,units_l1)*0.01
    b1=np.zeros((units_l2,1))
      
    W2=np.random.rand(units_l3, units_l2)*0.01
    b2=np.zeros((units_l3,1))
    
    
    W3=np.random.rand(units_l4, units_l3)*0.01
    b3=np.zeros((units_l4,1))
    
    m=len(X)
    
    def predict(X,W1, b1, W2, b2, W3, b3):
        ans=[]
        
        for i in range(len(X)):
            z2=np.dot(W1, X[i].reshape(-1,1))+b1

            a2=tangent(z2)
            z3=np.dot(W2,a2)+b2 
            a3=tangent(z3)
            z4=np.dot(W3,a3)+b3 
            a4=z4
            ans.append(a4)
            
        return ans
        
    for epoch in range(epochs):
        
        
        J0=0
        
        delW1=np.zeros((units_l2,units_l1))
        delb1=np.zeros((units_l2,1))
        delW2=np.zeros((units_l3, units_l2))
        delb2=np.zeros((units_l3,1))
        delW3=np.zeros((units_l4, units_l3))
        delb3=np.zeros((units_l4,1))
        
        for i in range(m):
            
            # forward
            z2=np.dot(W1, X[i].reshape(-1,1))+b1

            a2=tangent(z2)
            z3=np.dot(W2,a2)+b2 
            a3=tangent(z3)
            z4=np.dot(W3,a3)+b3 
#             a4=leaky_rectified_linear(z4)
            a4=z4
            
            J0+=0.5*(a4-y[i])**2
            
            # backward
            delta4=-(y[i]-a4)
            delta3=np.dot(W3.T,delta4)*(1-a3**2)
            delta2=np.dot(W2.T,delta3)*(1-a2**2)
            
            
            derivW3=np.dot(delta4, a3.T)
            derivb3=delta4
            
            derivW2=np.dot(delta3, a2.T)
            derivb2=delta3
            
            derivW1=np.dot(delta2, X[i].reshape(-1,1).T)
            derivb1=delta2
            
            delW3+=derivW3
            delb3+=derivb3
            
            delW2+=derivW2
            delb2+=derivb2

            delW1+=derivW1
            delb1+=derivb1

        W3=W3-learning_rate*(1/m*delW3+lambda_*W3)     
        b3=b3-learning_rate*(1/m*delb3)  
        
        W2=W2-learning_rate*(1/m*delW2+lambda_*W2)     
        b2=b2-learning_rate*(1/m*delb2)

        W1=W1-learning_rate*(1/m*delW1+lambda_*W1)     
        b1=b1-learning_rate*(1/m*delb1)
        
        
        J=(1/m)*J0+(lambda_/2)*((W1**2).sum(axis=1).sum(axis=0)
                                +(W2**2).sum(axis=1).sum(axis=0)
                                +(W3**2).sum(axis=1).sum(axis=0))
        
        if epoch%500==0:
            print("loss after {} epochs: ".format(epoch+1), J)
            y_pred=predict(X_test,W1, b1, W2, b2, W3, b3)
            print("score: ", r2_score(y_test, np.concatenate(np.concatenate(np.stack(y_pred,axis=0)))))
            print()
        
        
        
    return W1, b1, W2, b2, W3, b3

In [24]:
W1, b1, W2, b2, W3, b3 = MLP_4layers(X_train, y_train, epochs=20001, lambda_ =0.0005, learning_rate=0.001)

loss after 1 epochs:  [[22435.00733134]]
score:  -0.001418295980891049

loss after 501 epochs:  [[1742.80693614]]
score:  0.9144470018485368

loss after 1001 epochs:  [[639.74958407]]
score:  0.9690661905067783

loss after 1501 epochs:  [[359.79478189]]
score:  0.9832736883146312

loss after 2001 epochs:  [[242.03815886]]
score:  0.9888930004049797

loss after 2501 epochs:  [[174.4355137]]
score:  0.9917558933322074

loss after 3001 epochs:  [[133.98314805]]
score:  0.9936917397410251

loss after 3501 epochs:  [[110.21664909]]
score:  0.9950768460141864

loss after 4001 epochs:  [[92.52777265]]
score:  0.9958653669469345

loss after 4501 epochs:  [[78.17778672]]
score:  0.9965181403804687

loss after 5001 epochs:  [[67.82179928]]
score:  0.9970487695056517

loss after 5501 epochs:  [[59.22815767]]
score:  0.9975025422909696

loss after 6001 epochs:  [[52.23581673]]
score:  0.9978509479161983

loss after 6501 epochs:  [[46.89934531]]
score:  0.9981287716620304

loss after 7001 epochs:  

In [25]:
def predict_4layer(X, W1, b1, W2, b2, W3, b3 ):
    ans=[]
    for i in range(len(X)):
        z2=np.dot(W1, X[i].reshape(-1,1))+b1

        a2=tangent(z2)
        z3=np.dot(W2,a2)+b2 
        a3=tangent(z3)
        z4=np.dot(W3,a3)+b3 
#             a4=leaky_rectified_linear(z4)
        a4=z4
        ans.append(a4)
    
    return ans

In [26]:
y_pred=predict_4layer(X_test, W1, b1, W2, b2, W3, b3 )

In [27]:
r2_score(y_test, np.concatenate(np.concatenate(np.stack(y_pred,axis=0))))

0.998882671067041

In [28]:
mean_squared_error(y_test, np.concatenate(np.concatenate(np.stack(y_pred,axis=0))))

51.539695404964725